In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O horse_and_human.zip

--2020-07-07 10:46:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 74.125.24.128, 2404:6800:4003:c04::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse_and_human.zip’

horse_and_human.zip 100%[===================>] 142.65M  20.3MB/s    in 7.0s    

2020-07-07 10:47:07 (20.3 MB/s) - ‘horse_and_human.zip’ saved [149574867/149574867]



In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O validation-horse_and_human.zip

--2020-07-07 10:47:09--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 172.217.194.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘validation-horse_and_human.zip’

validation-horse_an 100%[===================>]  10.95M  23.9MB/s    in 0.5s    

2020-07-07 10:47:10 (23.9 MB/s) - ‘validation-horse_and_human.zip’ saved [11480187/11480187]



In [ ]:
class myCallbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')> 0.75):
      print('\n Reach 83% validation Accuracy')
      self.model.stop_training = True

In [ ]:
callbacks = myCallbacks()

In [ ]:
import os
os.listdir()

['.config',
 'validation-horse_and_human.zip',
 'horse_and_human.zip',
 'sample_data']

In [ ]:
myzip = 'horse_and_human.zip'
myzip_valid = 'validation-horse_and_human.zip'

In [ ]:
import zipfile

In [ ]:
zip_use = zipfile.ZipFile(myzip,'r')
zip_use.extractall('Horse_and_Human')
zip_use.close()

In [ ]:
os.listdir()

['.config',
 'Horse_and_Human',
 'validation-horse_and_human.zip',
 'horse_and_human.zip',
 'sample_data']

In [ ]:
zip_use2 = zipfile.ZipFile(myzip_valid,'r')
zip_use2.extractall('Horse_and_Human_Valid')
zip_use2.close()

In [ ]:
os.listdir()

['.config',
 'Horse_and_Human_Valid',
 'Horse_and_Human',
 'validation-horse_and_human.zip',
 'horse_and_human.zip',
 'sample_data']

In [ ]:
#ploting the images requires us to make the list of horses and humans file 

#this will be done later on rn we focus on the model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

In [ ]:
train_datagen = IDG(rescale=1/255.0,rotation_range=0.2,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,fill_mode='nearest')
validation_datagen = IDG(rescale=1/255.0)

In [ ]:
train_generator = train_datagen.flow_from_directory('Horse_and_Human',target_size=(300,300),class_mode='binary',batch_size=128)
validation_generator = validation_datagen.flow_from_directory('Horse_and_Human_Valid',target_size=(300,300),class_mode='binary',batch_size=32)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.RMSprop(lr=0.001),metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,steps_per_epoch=8,epochs=15,verbose=1,validation_data=validation_generator,validation_steps=8,callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 18s 2s/step - loss: 0.7057 - accuracy: 0.5484 - val_loss: 0.6852 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 22s 3s/step - loss: 0.8056 - accuracy: 0.5940 - val_loss: 0.6625 - val_accuracy: 0.6445
Epoch 3/15
8/8 [==============================] - 20s 2s/step - loss: 0.6279 - accuracy: 0.6785 - val_loss: 0.7354 - val_accuracy: 0.5000
Epoch 4/15
8/8 [==============================] - 22s 3s/step - loss: 0.7605 - accuracy: 0.6808 - val_loss: 1.7294 - val_accuracy: 0.5000
Epoch 5/15
8/8 [==============================] - 20s 2s/step - loss: 0.9931 - accuracy: 0.7008 - val_loss: 0.6296 - val_accuracy: 0.6758
Epoch 6/15
8/8 [==============================] - 19s 2s/step - loss: 0.5948 - accuracy: 0.7030 - val_loss: 1.2355 - val_accuracy: 0.5156
Epoch 7/15
8/8 [==============================] - ETA: 0s - loss: 0.4754 - accuracy: 0.7864
 Reach 83% validation Accuracy
8/8 [==============================] - 19s 2s/ste

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
uploaded = files.upload()
for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path,target_size=(300,300))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  images = np.vstack([x])
  classes = model.predict(images,batch_size=10)
  classes[0]
  print(classes)
  if classes[0] > 0.5:
    print(fn+' is a Human')
  else :
    print(fn+' is a Horse') 

Saving humanimage5.jpg to humanimage5.jpg
Saving humanimage4.jpg to humanimage4.jpg
Saving humanwomenimages3.jpg to humanwomenimages3.jpg
Saving humanimage2.jpg to humanimage2.jpg
Saving humanimage1.jpg to humanimage1.jpg
Saving horseimage3.jpg to horseimage3.jpg
Saving horseimage2.jpg to horseimage2.jpg
Saving horseimage1.jpg to horseimage1.jpg
[[0.]]
humanimage5.jpg is a Horse
[[1.]]
humanimage4.jpg is a Human
[[0.]]
humanwomenimages3.jpg is a Horse
[[1.]]
humanimage2.jpg is a Human
[[0.]]
humanimage1.jpg is a Horse
[[1.]]
horseimage3.jpg is a Human
[[0.]]
horseimage2.jpg is a Horse
[[0.]]
horseimage1.jpg is a Horse


In [ ]:
# 4 wrong 3 right 
#wonder what are we missing in here ..?
